In [2]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib tk
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
from ctypes import cast,POINTER
import comtypes
import scipy.io.wavfile as wav
import scipy.signal as sig
import math

In [3]:
%matplotlib tk

In [4]:
audioSample = 1024*4
Format = pyaudio.paInt16
channels = 2
rate = 44100
NPERSEG = 512
INT16_MAX = 32767
frameDelay = 0.04

In [5]:
def initialiseStream():
    p = pyaudio.PyAudio()
    stream = p.open(
                format=Format,
                channels=channels,
                rate=rate,
                input=True,
                frames_per_buffer=audioSample,
                input_device_index=2)
    return stream

def initialiseFigures(portraitOrLandscape = 'p'):
    if portraitOrLandscape == 'p' or portraitOrLandscape == 'P':
        fig,axes = plt.subplots(3, 1)
    else:
        fig,axes = plt.subplots(1,3)

    axes[0].set_xlim(0, audioSample)
    axes[0].set_ylim(-0.15,0.15)
    axes[1].set_ylim([0,150])
    axes[0].axis('off')
    axes[1].axis('off')
    axes[2].axis('off')
    axes[2].set_ylim([0,5000])
    return fig,axes

def getData(stream):
    data = np.reshape(
                      np.frombuffer(stream.read(audioSample),dtype=np.int16),
                      (audioSample,channels)
    )
    return data

def getTransformedData(dataChannel):
    ex = np.fft.rfft(dataChannel)
    freqs = np.fft.fftfreq(ex.size)
    mag = np.abs(ex)
    return freqs,mag

def canvasReset():
    fig.canvas.draw()
    fig.canvas.flush_events()

In [6]:
fig,axes = initialiseFigures('p')
x = np.arange(0,2*audioSample,2)
line, = axes[0].plot(x,np.random.rand(audioSample))
line2, = axes[0].plot(x,np.random.rand(audioSample), color='orange')
stream = initialiseStream()
dayta = getData(stream)
f,t,Sxx = sig.spectrogram(dayta[:,0], fs=rate, window="hann",
                              nperseg=NPERSEG)
y = axes[2].pcolormesh(t, f, 10*np.log10(Sxx), vmin=-20, vmax=60)
freqs,mag = getTransformedData(dayta[:,0])
line3, = axes[1].plot(np.fft.fftshift(freqs),10*np.log10(mag))
plt.ion()
plt.show()
counter = 0 
startTime = time.time()
while True:
    time.sleep(frameDelay)
    dayta = getData(stream)
    freqs,mag = getTransformedData(dayta[:,0])
    f,t,Sxx = sig.spectrogram(dayta[:,0], fs=rate, window="hann",
                              nperseg=NPERSEG)
    axes[0].set_ylim(-np.max(dayta)/INT16_MAX,np.max(dayta)/INT16_MAX)
    axes[1].set_ylim(0,np.max(10*np.log10(np.abs(mag))))
    line.set_ydata(dayta[:,0]/INT16_MAX)
    line2.set_ydata(dayta[:,1]/INT16_MAX)
    line3.set_data([np.fft.fftshift(freqs)],[10*np.log10(np.maximum(np.abs(mag), 1e-10))])
    y.set_array(10*np.log10(Sxx).ravel())
    canvasReset()
    #FPS Counter.
    counter += 1
    if (time.time() - startTime) > 10:
        print('FPS: ',counter/(time.time()-startTime) )
        break



TclError: invalid command name "pyimage10"